<a href="https://colab.research.google.com/github/hanguyenhant/PLSA/blob/master/PLSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/PLSA')

In [0]:
!ls /content/gdrive/My\ Drive/PLSA

data_vectorizer.txt  PLSA.ipynb  vocab.txt


In [0]:
import numpy as np 
import time

In [0]:
def prepare_data(path):
	K = 5 #number_of_topics

	with open('/content/gdrive/My Drive/PLSA/vocab.txt', errors = 'ignore') as f:
		lines = f.read().splitlines() #number_of_words_in_vocab

	V = len(lines)+1

	with open(path, errors = 'ignore') as f:
		lines = f.read().splitlines()

	D = len(lines) #number_of_docs

	doc_count = []

	for line in lines:

		word_count = dict()
		words = line.split('<fff>')[2].split()

		for word in words:
			word_count[int(word.split(':')[0])] = int(word.split(':')[1])

		doc_count.append(word_count) #doc_count có dạng: [{10:2, 12:3}, {11:3, 13:1, 12:4}, {4:2, 2:4, 1:2, 5:2}]

	return K, V, D, doc_count

In [0]:
class PLSA:
	def __init__(self, K, V, D, doc_count):

		self._K = K #number_of_topics
		self._V = V #number_of_words_in_vocab
		self._D = D #number_of_docs
		self._doc_count = doc_count

		#X = wordcountPerdoc
		self._X = np.zeros(shape = (D, V))
		for i, d in enumerate(self._doc_count):
			for key, value in d.items():
				self._X[i][key] = value

		self._theta = np.random.random(size = (D, K))
		self._beta = np.random.random(size = (K, V))
		self._T = np.zeros(shape = (D, V, K))

	def normalize(self):
		for d in range(self._D):

			normalization = np.sum(self._theta[d, :])

			for k in range(self._K):
				self._theta[d, k] /= normalization

		for k in range(self._K):

			normalization = np.sum(self._beta[k, :])

			for v in range(self._V):
				self._beta[k, v] /= normalization

	def E_step(self):
		for d in range(self._D):
			for v in range(self._V):

				denominator = 0

				for k in range(self._K):
					self._T[d, v, k] = self._theta[d, k] * self._beta[k, v]
					denominator += self._T[d, v, k]

				if denominator == 0:
					for k in range(self._K):
						self._T[d, v, k] = 0;
				else:
					for k in range(self._K):
						self._T[d, v, k] /= denominator;

	def M_step(self):
		#Tinh theta
		for d in range(self._D):
			for k in range(self._K):

				self._theta[d, k] = 0
				denominator = 0

				for v in range(self._V):
					self._theta[d, k] += self._X[d, v] * self._T[d, v, k]
					denominator += self._X[d, v]


				if denominator == 0:
					self._theta[d, k] = 1.0 / K
				else:
					self._theta[d, k] /= denominator

		#Tinh beta
		for k in range(self._K):

			denominator = 0
			print('K: ', k)

			for v in range(self._V):

				self._beta[k, v] = 0

				for d in range(self._D):
					self._beta[k, v] += self._X[d, v] * self._T[d, v, k]

				denominator += self._beta[k, v]

			if denominator == 0:
				for v in range(self._V):
					self._beta[k, v] = 1.0 / self._V
			else:
				for v in range(self._V):
					self._beta[k, v] /= denominator

	def LogLikelihood(self):
		loglikelihood = 0

		for d in range(self._D):
			for v in range(self._V):
				tmp = 0
				for k in range(self._K):
					tmp += self._theta[d, k] * self._beta[k, v]

				if tmp > 0:
					loglikelihood += self._X[d, v] * np.log(tmp)
		return loglikelihood

In [0]:
K, V, D, doc_count = prepare_data('/content/gdrive/My Drive/PLSA/data_vectorizer.txt')

print('K, V, D', K, V, D)
plsa = PLSA(K, V, D, doc_count)
plsa.normalize()

L = 0

for i in range(100):
  plsa.E_step()
  print('E_step lan ', i+1)
  plsa.M_step()
  print('M_step lan ', i+1)
  print("[", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), "] After the", i+1, "'s iteration  ", )
  L_new = plsa.LogLikelihood()
  print('Log Likelihood: ', L_new)
  print('Loss: ', abs(L - L_new))
  if abs(L - L_new) <= 10e-4:
    break
  else:
    L = L_new

K, V, D 5 5565 249
E_step lan  1
K:  0
K:  1
K:  2
K:  3
K:  4
M_step lan  1
[ 2019-07-15 01:19:41 ] After the 1 's iteration  
Log Likelihood:  -99349.01466508296
Loss:  99349.01466508296
E_step lan  2
K:  0
K:  1
K:  2
K:  3
K:  4
M_step lan  2
[ 2019-07-15 01:20:12 ] After the 2 's iteration  
Log Likelihood:  -97595.08953603255
Loss:  1753.9251290504035
E_step lan  3
K:  0
K:  1
K:  2
K:  3
K:  4
M_step lan  3
[ 2019-07-15 01:20:42 ] After the 3 's iteration  
Log Likelihood:  -95265.83465088146
Loss:  2329.2548851510946
E_step lan  4
K:  0
K:  1
K:  2
K:  3
K:  4
M_step lan  4
[ 2019-07-15 01:21:13 ] After the 4 's iteration  
Log Likelihood:  -92871.84599013095
Loss:  2393.988660750503
E_step lan  5
K:  0
K:  1
K:  2
K:  3
K:  4
M_step lan  5
[ 2019-07-15 01:21:44 ] After the 5 's iteration  
Log Likelihood:  -90834.72988878695
Loss:  2037.1161013439996
E_step lan  6
K:  0
K:  1
K:  2
K:  3
K:  4
M_step lan  6
[ 2019-07-15 01:22:14 ] After the 6 's iteration  
Log Likelihood:  -8

In [0]:
print(plsa._theta)

[[6.54939525e-087 7.19052202e-061 1.00000000e+000 1.43961266e-074
  1.06031844e-084]
 [9.48452747e-073 1.00000000e+000 4.81719675e-068 9.36475113e-097
  2.70784259e-076]
 [4.52386743e-067 2.77143499e-042 1.00000000e+000 1.56971064e-058
  4.76927839e-095]
 ...
 [2.56031739e-091 5.89616272e-082 2.19259480e-019 4.02875283e-053
  1.00000000e+000]
 [1.27357806e-112 2.38563675e-131 5.60307235e-140 1.14320365e-114
  1.00000000e+000]
 [4.13367425e-124 1.68635590e-120 2.19952380e-093 1.00000000e+000
  6.33617693e-076]]


In [0]:
plsa._theta.shape

(249, 5)

In [0]:
np.argsort(plsa._theta)[:, -1]

array([2, 1, 2, 0, 0, 1, 4, 1, 0, 4, 4, 3, 0, 1, 4, 2, 4, 4, 4, 1, 0, 0,
       0, 4, 0, 4, 3, 3, 3, 3, 4, 4, 4, 2, 4, 0, 0, 0, 3, 1, 4, 0, 0, 3,
       4, 1, 2, 0, 0, 0, 2, 3, 0, 4, 0, 0, 4, 0, 0, 4, 3, 0, 4, 2, 1, 3,
       2, 3, 4, 1, 2, 1, 2, 3, 3, 3, 0, 4, 2, 3, 2, 0, 4, 3, 2, 0, 2, 2,
       3, 3, 2, 3, 3, 2, 1, 0, 3, 0, 1, 0, 1, 2, 0, 2, 0, 4, 0, 3, 0, 3,
       1, 4, 0, 1, 0, 3, 3, 1, 0, 4, 3, 4, 3, 2, 4, 3, 2, 1, 2, 0, 2, 0,
       3, 1, 0, 3, 1, 2, 3, 0, 4, 0, 1, 3, 4, 3, 1, 4, 4, 0, 3, 3, 3, 1,
       0, 4, 4, 4, 0, 3, 1, 4, 1, 4, 4, 0, 0, 2, 0, 3, 4, 3, 4, 1, 4, 3,
       0, 1, 2, 1, 1, 1, 0, 1, 4, 2, 0, 4, 2, 0, 4, 2, 4, 1, 2, 4, 0, 1,
       1, 1, 2, 2, 3, 1, 0, 0, 0, 0, 3, 4, 0, 0, 4, 1, 1, 4, 4, 4, 3, 1,
       1, 3, 3, 1, 3, 4, 0, 1, 4, 1, 4, 3, 1, 3, 4, 0, 2, 1, 4, 2, 0, 3,
       1, 0, 2, 1, 4, 4, 3])

In [0]:
print(plsa._beta)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.0007073  0.         0.00035365]
 [0.00038754 0.00038754 0.00077507 ... 0.         0.         0.00038754]
 [0.         0.         0.         ... 0.         0.00103508 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [0]:
np.argsort(plsa._beta)[:, -10:]

array([[2605, 5318, 3271, 4701, 5330, 3190, 3830, 4430, 4859, 2855],
       [3237,  235, 3382, 3964,  427, 2178, 4341, 4144, 5434, 5124],
       [3563, 3094, 2660, 2531, 3469, 1921, 2204, 1152, 1878, 4283],
       [4469, 4926, 4047, 3468, 3977, 5508, 3210, 1537, 2690, 5538],
       [ 605, 4528, 5355, 5551, 1340, 4324, 2827,  762, 2744, 1630]])

In [0]:
with open('/content/gdrive/My Drive/PLSA/vocab.txt', errors = 'ignore') as f:
  vocab = f.read().splitlines()
  

In [0]:
vocab

['aa',
 'aah',
 'aap',
 'aaron',
 'ab',
 'abortions',
 'absence',
 'absolute',
 'abstact',
 'abstract',
 'abstracts',
 'absurd',
 'absurdities',
 'abyte',
 'academic',
 'accelerated',
 'accelerators',
 'accellerator',
 'acceptable',
 'acceptance',
 'accepting',
 'accepts',
 'accessed',
 'accessible',
 'accident',
 'accidental',
 'accidents',
 'accompanied',
 'accomplishment',
 'accor',
 'accounts',
 'accurately',
 'achieve',
 'acooper',
 'acquired',
 'acquisition',
 'across',
 'action',
 'actions',
 'active',
 'actively',
 'activity',
 'acts',
 'actual',
 'ad',
 'adam',
 'adams',
 'adaptabtion',
 'adaptec',
 'adapters',
 'adb',
 'added',
 'addendum',
 'adding',
 'addition',
 'addressed',
 'addressing',
 'adequate',
 'adequately',
 'adherence',
 'adherent',
 'adjusted',
 'administer',
 'administers',
 'administrative',
 'administrator',
 'admit',
 'admitted',
 'admittedly',
 'admitting',
 'adobe',
 'adopt',
 'adresses',
 'adriene',
 'ads',
 'adult',
 'adulteries',
 'advanced',
 'advanta

In [0]:
k = 0
for t in np.argsort(plsa._beta)[:, -10:]:
  print('Topic ', k)
  print('-----')
  for w in t:
    print(vocab[w])
  k += 1
  print('*****')

Topic  0
-----
jason
virtual
navy
sta
visualization
morality
presentations
seminar
swap
lipman
*****
Topic  1
-----
muslims
arbor
nutek
purchase
believers
harddisk
satan
religions
wide
tt
*****
Topic  2
-----
parameters
mfm
jumper
interleave
optional
formatting
heads
cylinders
floppy
rom
*****
Topic  3
-----
sg
td
rd
option
q
wsm
movie
edge
keyboard
yap
*****
Topic  4
-----
cable
signal
vram
youth
diamond
sale
lex
cie
la
enviroleague
*****
